In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2
import numpy as np
import pickle
from numpy.linalg import norm

# Load the preprocessed features and filenames
resnet50_features = np.array(pickle.load(open('pickle files/resnet50_features.pkl', 'rb')))
color_features = np.array(pickle.load(open('pickle files/color_features.pkl', 'rb')))
filenames = pickle.load(open('pickle files/filenames.pkl', 'rb'))

# Combine both features
combined_features = np.hstack((resnet50_features, color_features))

# Define the model for feature extraction
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False
model = tf.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

def save_uploaded_file(uploaded_file_path):
    try:
        if not os.path.exists('uploads'):
            os.makedirs('uploads')
        base_filename = os.path.basename(uploaded_file_path)
        dest_path = os.path.join('uploads', base_filename)
        if not os.path.exists(dest_path):
            with open(dest_path, 'wb') as f:
                with open(uploaded_file_path, 'rb') as uf:
                    f.write(uf.read())
        return dest_path
    except Exception as e:
        print(e)
        return None

def extract_resnet50_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    return normalized_result

def extract_color_histogram(image_path, bins=(8, 8, 8)):
    img = cv2.imread(image_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def feature_extraction(img_path, model):
    resnet50_feature = extract_resnet50_features(img_path, model)
    color_feature = extract_color_histogram(img_path)
    combined_feature = np.hstack((resnet50_feature, color_feature))
    return combined_feature

def recommend(features, feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)
    distances, indices = neighbors.kneighbors([features])
    return indices

# Upload your image
uploaded_file_name = input("Enter the path of the image: ")
if os.path.exists(uploaded_file_name):
    saved_file_path = save_uploaded_file(uploaded_file_name)
    if saved_file_path:
        features = feature_extraction(saved_file_path, model)
        indices = recommend(features, combined_features)

        print("Recommended images:")
        for i in range(len(indices[0])):
            img_path = filenames[indices[0][i]]
     
            print(f"Displaying: {img_path}")
            img = cv2.imread(img_path)
            resized_img = cv2.resize(img, (300, 300))
            cv2.imshow(f'Recommended Image {i + 1}', resized_img)

        cv2.waitKey(0)
        cv2.destroyAllWindows()
else:
    print("File does not exist.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Recommended images:
Displaying: preprocessed_images\22629.jpg
Displaying: preprocessed_images\2310.jpg
Displaying: preprocessed_images\1644.jpg
Displaying: preprocessed_images\5433.jpg
Displaying: preprocessed_images\22597.jpg
Displaying: preprocessed_images\13266.jpg
